## Naive Bayes on Mobility

In [1]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
# Import the dataset for mobility
mobility = pd.read_csv("../data/usitc/border_crossing_entry_data_trade.csv")

In [3]:
mobility.head()

,Port Name,State,Port Code,Border,Date,Measure,Value,imports,exports
0,Alexandria Bay,NY,708,US-Canada Border,12/1/2019 0:00,Bus Passengers,740,NaN,NaN
1,Baudette,MN,3424,US-Canada Border,12/1/2019 0:00,Bus Passengers,83,NaN,NaN
2,Blaine,WA,3004,US-Canada Border,12/1/2019 0:00,Bus Passengers,17029,NaN,NaN
3,Brownsville,TX,2301,US-Mexico Border,12/1/2019 0:00,Bus Passengers,4668,3.580000e+11,1.960000e+11
4,Buffalo-Niagara Falls,NY,901,US-Canada Border,12/1/2019 0:00,Bus Passengers,9910,NaN,NaN


In [4]:
# Include only mobility on the US-MX border
borderMX = mobility[(mobility['Border']=="US-Mexico Border")]

In [ ]:
borderMX.head()

,Port Name,State,Port Code,Border,Date,Measure,Value,imports,exports
3,Brownsville,TX,2301,US-Mexico Border,12/1/2019 0:00,Bus Passengers,4668,3.580000e+11,1.960000e+11
6,Calexico East,CA,2507,US-Mexico Border,12/1/2019 0:00,Bus Passengers,4080,3.580000e+11,1.960000e+11
8,Columbus,NM,2406,US-Mexico Border,12/1/2019 0:00,Bus Passengers,1452,3.580000e+11,1.960000e+11
11,Douglas,AZ,2601,US-Mexico Border,12/1/2019 0:00,Bus Passengers,107,3.580000e+11,1.960000e+11
13,Eagle Pass,TX,2303,US-Mexico Border,12/1/2019 0:00,Bus Passengers,8525,3.580000e+11,1.960000e+11


In [ ]:
# Extract year and consolidate year
import datetime
borderMX['year']= pd.DatetimeIndex(borderMX['Date']).year

C:\Users\nhinojos\AppData\Local\Continuum\anaconda\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


## Encoding the Categorical Variable State

In [ ]:
df = borderMX[["year","State","Port Code","Value"]].copy()

In [ ]:
X1 = df.iloc[:,:-1].values
y = df.iloc[:,3].values

In [ ]:
# Encoding categorical data
# Encoding the independent variable
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [ ]:
labelencoder = LabelEncoder()
X1[:, 1] = labelencoder.fit_transform(X1[:, 1])
onehotencoder = OneHotEncoder(categorical_features = [1])
X1 = onehotencoder.fit_transform(X1).toarray()

C:\Users\nhinojos\AppData\Local\Continuum\anaconda\lib\site-packages\sklearn\preprocessing\_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
C:\Users\nhinojos\AppData\Local\Continuum\anaconda\lib\site-packages\sklearn\preprocessing\_encoders.py:390: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


In [ ]:
# California is Index 1, KEEP
print(X1.shape)

(82288, 6)


In [ ]:
print(X1)

[[0.000e+00 0.000e+00 0.000e+00 1.000e+00 2.019e+03 2.301e+03]
 [0.000e+00 1.000e+00 0.000e+00 0.000e+00 2.019e+03 2.507e+03]
 [0.000e+00 0.000e+00 1.000e+00 0.000e+00 2.019e+03 2.406e+03]
 ...
 [0.000e+00 0.000e+00 1.000e+00 0.000e+00 1.996e+03 2.408e+03]
 [0.000e+00 0.000e+00 0.000e+00 1.000e+00 1.996e+03 2.403e+03]
 [0.000e+00 1.000e+00 0.000e+00 0.000e+00 1.996e+03 2.505e+03]]


In [ ]:
# Re-establishing the X matrix
X = X1[:,[1,4,5]]
print(X)

[[0.000e+00 2.019e+03 2.301e+03]
 [1.000e+00 2.019e+03 2.507e+03]
 [0.000e+00 2.019e+03 2.406e+03]
 ...
 [0.000e+00 1.996e+03 2.408e+03]
 [0.000e+00 1.996e+03 2.403e+03]
 [1.000e+00 1.996e+03 2.505e+03]]


## Starting Naive Bayes

In [ ]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)


In [ ]:
# Fitting Naive Bayes to the Training set
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X_train, y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [ ]:
# Predicting the Test set results
y_pred = classifier.predict(X_test)

In [ ]:
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

In [ ]:
# Visualizing the Training set results
from matplotlib.colors import ListedColormap
X_set, y_set = X_train, y_train
X1, X2 = np.meshgrid(np.arange(start = X_set[:, 0].min() - 1, stop = X_set[:, 0].max() + 1, step = 0.01),
                     np.arange(start = X_set[:, 1].min() - 1, stop = X_set[:, 1].max() + 1, step = 0.01))
plt.contourf(X1, X2, classifier.predict(np.array([X1.ravel(), X2.ravel()]).T).reshape(X1.shape),
             alpha = 0.75, cmap = ListedColormap(('red', 'green')))
plt.xlim(X1.min(), X1.max())
plt.ylim(X2.min(), X2.max())
for i, j in enumerate(np.unique(y_set)):
    plt.scatter(X_set[y_set == j, 0], X_set[y_set == j, 1],
                c = ListedColormap(('red', 'green'))(i), label = j)
plt.title('Naive Bayes (Training set)')
plt.xlabel('Age')
plt.ylabel('Estimated Salary')
plt.legend()
plt.show()

In [ ]:

# Visualising the Test set results
from matplotlib.colors import ListedColormap
X_set, y_set = X_test, y_test
X1, X2 = np.meshgrid(np.arange(start = X_set[:, 0].min() - 1, stop = X_set[:, 0].max() + 1, step = 0.01),
                     np.arange(start = X_set[:, 1].min() - 1, stop = X_set[:, 1].max() + 1, step = 0.01))
plt.contourf(X1, X2, classifier.predict(np.array([X1.ravel(), X2.ravel()]).T).reshape(X1.shape),
             alpha = 0.75, cmap = ListedColormap(('red', 'green')))
plt.xlim(X1.min(), X1.max())
plt.ylim(X2.min(), X2.max())
for i, j in enumerate(np.unique(y_set)):
    plt.scatter(X_set[y_set == j, 0], X_set[y_set == j, 1],
                c = ListedColormap(('red', 'green'))(i), label = j)
plt.title('Naive Bayes (Test set)')
plt.xlabel('Age')
plt.ylabel('Estimated Salary')
plt.legend()
plt.show()

In [ ]:
print("hello")

In [ ]:
print("hello!")